In [1]:
from bs4 import BeautifulSoup
import getNDVI
import getSAR
import common
import numpy as np
import ee
import getPair
from datetime import datetime
import disp_multiple_images

%matplotlib inline

In [2]:
# Trigger the authentication flow.
#ee.Authenticate()

In [3]:
# Initialize the library.
ee.Initialize()

In [4]:
def parseKML(inputfile):
  with open(inputfile, "r") as f:
    soup = BeautifulSoup(f, features="html.parser")
    return [np.array([i.split(",")[0:2] for i in node.string.split()]).astype(np.float).tolist()\
            for node in soup.findAll("coordinates")]

In [7]:
if __name__ == "__main__":
  for array in parseKML("2017_polygons.kml"):
    startDate, EndDate = "2017-01-01","2017-02-01"    
    data, _ = getPair.arrayToPairs(array, startDate, EndDate, False, 0.5)
    
    px_pairs = None
    for i in range(0, len(data), 2):
        # NDVI, SAR
        tmp = np.dstack((data[i][0], data[i+1][0]))
        tmp = tmp.reshape(-1, tmp.shape[-1])
        tmp = tmp[np.all(tmp != 0, axis=1)]
        px_pairs = np.concatenate((px_pairs, tmp), axis = 0) if not px_pairs is None else tmp
    
    # px_pairs is now pixel-pairs for a given field over a given time span
    
    break

{0: [0], 1: [0], 2: [0], 3: [0], 4: [1], 5: [1]}
[[ 6.16822429e-02 -1.91848335e+01]
 [ 2.78884470e-02 -1.94849224e+01]
 [ 9.11854114e-03 -1.81715927e+01]
 [-2.69865058e-02 -1.72241764e+01]
 [ 1.66305136e-02 -1.73056335e+01]
 [ 6.22986034e-02 -2.06316624e+01]
 [ 4.16415222e-02 -1.97951717e+01]
 [ 7.24637695e-03 -1.86536274e+01]
 [-2.43902430e-02 -1.84338017e+01]
 [-7.06991367e-03 -1.84697132e+01]
 [ 7.54716981e-04 -1.73744125e+01]
 [-1.31274136e-02 -1.49093952e+01]
 [ 1.90217383e-02 -1.45739822e+01]
 [ 4.24615368e-02 -1.36849613e+01]
 [ 4.99698967e-02 -1.28717871e+01]
 [ 3.59628759e-02 -1.33823824e+01]
 [ 5.42035401e-02 -1.48260393e+01]
 [ 7.43353218e-02 -1.65426502e+01]
 [ 4.42708321e-02 -1.88993416e+01]
 [ 3.21257673e-02 -1.75129318e+01]
 [-1.10803321e-02 -1.58185940e+01]
 [ 9.31899622e-03 -1.60066204e+01]
 [ 1.79211479e-02 -1.72316399e+01]
 [ 5.97101450e-02 -1.67442970e+01]
 [ 7.30619058e-02 -1.72111092e+01]
 [ 5.58937453e-02 -1.75210648e+01]
 [ 6.57402426e-02 -1.70685215e+01]
 [ 5.1